In [55]:
import arcpy
import json
import pandas as pd
import numpy as np
import requests
import time as t #datetime and time do not work together by default, change name of time to t
from datetime import datetime, timedelta

In [56]:
url = 'https://qaqcmilktruck-qfcu3qoavq-uc.a.run.app/5572/milk_truck_sensor'

#keep track of how many times we call the API
total_calls = 0

#keep track of errors
errors = 0

#Storing data that is good :)
final_data = []

#Makes the API call
response = requests.get(url)

In [57]:
response

<Response [200]>

In [58]:
#Check if we are good, 200
if response.status_code == 200:
    # Convert the JSON response to a dictionary (it allows us to read it in python easily)
    data = response.json()
    print("Data fetched successfully:", data)
else:
    #If the response is not 200, give an error and do nothing with the data
    print("Failed to fetch data. Status code:", response.status_code)

Data fetched successfully: {'lat': 45.755296749186726, 'lon': -94.76667309526766, 'current_time': '02/23/24:01:18:48', 'air_temperature': 38.16855882856333}


In [59]:
#This is the completeness check

#Check if any of our data is 'nan'
## True if the data is 'nan', False if it is not 'nan'
### We want all of these to be False!
latitude_error = data['lat'] == 'nan'
longitude_error = data['lon'] == 'nan'
time_error = data['current_time'] == 'nan'
temp_error = data['air_temperature'] == 'nan'

#If any of our items is True (there is an nan), then there is an error
if latitude_error or longitude_error or time_error or temp_error:
    #print there is an error
    print('error, missing data')
    #Add one to the errors
    errors = errors + 1
else:
    #All good if there are no errors!
    print('no missing data, continuing')

no missing data, continuing


In [60]:
#This is completeness check

#Boundaries of Minnesota
minnesota_north = 49.5
minnesota_east = -89.0
minnesota_south = 43.0
minnesota_west = -97.5

#If our data falls within Minnesota vertically
if data['lat'] < minnesota_north and data['lat'] > minnesota_south:
    print('Data within Minnesota vertically!')

#Error, it is outside Minnesota
else:
    #print there is an error
    print('error, latitude out of range')
    #Add one to the errors
    errors = errors + 1

#If our data falls within Minnesota horizontally
if data['lon'] > minnesota_west and data['lon'] < minnesota_east:
    print('Data within Minnesota horizontally!')

#Error, it is outside Minnesota
else:
    #print there is an error
    print('error, longitude out of range')
    #Add one to the errors
    errors = errors + 1


Data within Minnesota vertically!
Data within Minnesota horizontally!


In [61]:
#Thematic check +/- 20 degrees

min_temp = 11.0
max_temp = 51.0

#Check if temperature is above the minimum temperature AND lower than the maximum
if data['air_temperature'] > min_temp and data['air_temperature'] < max_temp:
    print('Temperature is slay!')
    
#Error, it is wrong temperature
else:
    #print there is an error
    print('error, temperature out of range... milk dont drink')
    #Add one to the errors
    errors = errors + 1

Temperature is slay!


In [62]:
#Time check
time = datetime.strptime(data['current_time'], '%m/%d/%y:%H:%M:%S')

#Get what time it is now
now = datetime.now()

#Check if time is within 24 hours from what time it is right now
if now-timedelta(hours=24) <= time <= now+timedelta(hours=24):
    print('Time is slay!')
#Error, it is wrong time
else:
    #print there is an error
    print('error, time is not slay')
    #Add one to the errors
    errors = errors + 1

Time is slay!


In [63]:
#Storing data in MEMORY as a python array
##This is temporary and will disappear if we close the program
##A better solution would save data to CSV
final_data.append(data)

In [64]:
final_data

[{'lat': 45.755296749186726, 'lon': -94.76667309526766, 'current_time': '02/23/24:01:18:48', 'air_temperature': 38.16855882856333}]

In [65]:
#Reset everything before we run our loop
url = 'https://qaqcmilktruck-qfcu3qoavq-uc.a.run.app/5572/milk_truck_sensor'

#keep track of how many times we call the API
total_calls = 0

#keep track of errors
errors = 0

#Storing data that is good :)
final_data = []


##Put everything else into a loop

#Run this loop 100 times, that is so slay
## whenever an error happens, there will be a continue, that restarts the loop from the beginining
for i in range(100):

    #Makes the API call
    response = requests.get(url)
    
    #Keep track of how many times we call our API
    total_calls = total_calls + 1 
    
    #Check if we are good, 200
    if response.status_code == 200:
        # Convert the JSON response to a dictionary (it allows us to read it in python easily)
        data = response.json()
        print("Data fetched successfully:", data)
    else:
        #If the response is not 200, give an error and do nothing with the data
        print("Failed to fetch data. Status code:", response.status_code)
        
        #If there are ANY Errors, restart the for loop
        continue
        
    #This is the completeness check

    #Check if any of our data is 'nan'
    ## True if the data is 'nan', False if it is not 'nan'
    ### We want all of these to be False!
    latitude_error = data['lat'] == 'nan'
    longitude_error = data['lon'] == 'nan'
    time_error = data['current_time'] == 'nan'
    temp_error = data['air_temperature'] == 'nan'

    #If any of our items is True (there is an nan), then there is an error
    if latitude_error or longitude_error or time_error or temp_error:
        #print there is an error
        print('error, missing data')
        #Add one to the errors
        errors = errors + 1
        continue
        
    else:
        #All good if there are no errors!
        print('no missing data, continuing')
        
    #This is completeness check

    #Boundaries of Minnesota
    minnesota_north = 49.5
    minnesota_east = -89.0
    minnesota_south = 43.0
    minnesota_west = -97.5

    #If our data falls within Minnesota vertically
    if data['lat'] < minnesota_north and data['lat'] > minnesota_south:
        print('Data within Minnesota vertically!')

    #Error, it is outside Minnesota
    else:
        #print there is an error
        print('error, latitude out of range')
        #Add one to the errors
        errors = errors + 1
        continue

    #If our data falls within Minnesota horizontally
    if data['lon'] > minnesota_west and data['lon'] < minnesota_east:
        print('Data within Minnesota horizontally!')

    #Error, it is outside Minnesota
    else:
        #print there is an error
        print('error, longitude out of range')
        #Add one to the errors
        errors = errors + 1
        continue

        
    #Thematic check +/- 20 degrees
    min_temp = 11.0
    max_temp = 51.0

    #Check if temperature is above the minimum temperature AND lower than the maximum
    if data['air_temperature'] > min_temp and data['air_temperature'] < max_temp:
        print('Temperature is slay!')

    #Error, it is wrong temperature
    else:
        #print there is an error
        print('error, temperature out of range... milk dont drink')
        #Add one to the errors
        errors = errors + 1
        continue
        
    #Time check
    time = datetime.strptime(data['current_time'], '%m/%d/%y:%H:%M:%S')

    now = datetime.now()

    #Check if time is within 24 hours from what time it is right now
    if now-timedelta(hours=24) <= time <= now+timedelta(hours=24):
        print('Time is slay!')
    #Error, it is wrong time
    else:
        #print there is an error
        print('error, time is not slay')
        #Add one to the errors
        errors = errors + 1
        continue
       

    #Storing data

    final_data.append(data)

    #Pause at the end for 0.1 second
    t.sleep(0.1)

Data fetched successfully: {'lat': 46.792283865223546, 'lon': -93.67735204559952, 'current_time': '02/23/24:01:18:50', 'air_temperature': 19.55294158064669}
no missing data, continuing
Data within Minnesota vertically!
Data within Minnesota horizontally!
Temperature is slay!
Time is slay!
Data fetched successfully: {'lat': 48.6544609690473, 'lon': -93.83973396416542, 'current_time': '01/21/29:15:11:12', 'air_temperature': 19.69201154864445}
no missing data, continuing
Data within Minnesota vertically!
Data within Minnesota horizontally!
Temperature is slay!
error, time is not slay
Data fetched successfully: {'lat': 46.22123560128494, 'lon': -94.8771750165459, 'current_time': '02/23/24:01:18:51', 'air_temperature': 37.50829940509462}
no missing data, continuing
Data within Minnesota vertically!
Data within Minnesota horizontally!
Temperature is slay!
Time is slay!
Data fetched successfully: {'lat': 'nan', 'lon': -93.9936928883378, 'current_time': '02/23/24:01:18:52', 'air_temperature': 

Data within Minnesota vertically!
Data within Minnesota horizontally!
Temperature is slay!
Time is slay!
Data fetched successfully: {'lat': 47.04125859628829, 'lon': -93.58514257171261, 'current_time': '02/23/24:01:19:03', 'air_temperature': 11.08059371208746}
no missing data, continuing
Data within Minnesota vertically!
Data within Minnesota horizontally!
Temperature is slay!
Time is slay!
Data fetched successfully: {'lat': 47.948400632444816, 'lon': -94.05404372463498, 'current_time': '02/23/24:01:19:03', 'air_temperature': 63.274587019352}
no missing data, continuing
Data within Minnesota vertically!
Data within Minnesota horizontally!
error, temperature out of range... milk dont drink
Data fetched successfully: {'lat': 45.90510974992367, 'lon': -94.48471409275525, 'current_time': '02/23/24:01:19:04', 'air_temperature': 47.121118429177}
no missing data, continuing
Data within Minnesota vertically!
Data within Minnesota horizontally!
Temperature is slay!
Time is slay!
Data fetched su

Data fetched successfully: {'lat': 49.112283152032276, 'lon': -97.59059822563785, 'current_time': '02/23/24:01:19:15', 'air_temperature': 35.06835911341315}
no missing data, continuing
Data within Minnesota vertically!
error, longitude out of range
Data fetched successfully: {'lat': 48.04859429829107, 'lon': -92.98987987501965, 'current_time': '02/23/24:01:19:16', 'air_temperature': 47.58982403758665}
no missing data, continuing
Data within Minnesota vertically!
Data within Minnesota horizontally!
Temperature is slay!
Time is slay!
Data fetched successfully: {'lat': 46.92836134030712, 'lon': -94.39249777485392, 'current_time': '02/23/24:01:19:16', 'air_temperature': 35.103232079903265}
no missing data, continuing
Data within Minnesota vertically!
Data within Minnesota horizontally!
Temperature is slay!
Time is slay!
Data fetched successfully: {'lat': 45.85845960190695, 'lon': -94.15487462739407, 'current_time': '01/21/29:15:11:12', 'air_temperature': 49.51451260779983}
no missing data,

error, temperature out of range... milk dont drink
Data fetched successfully: {'lat': 'nan', 'lon': -92.5744152997851, 'current_time': '02/23/24:01:19:27', 'air_temperature': 39.82860534356782}
error, missing data
Data fetched successfully: {'lat': 46.19832689264884, 'lon': -94.39566169211102, 'current_time': '02/23/24:01:19:28', 'air_temperature': 33.37071680282446}
no missing data, continuing
Data within Minnesota vertically!
Data within Minnesota horizontally!
Temperature is slay!
Time is slay!
Data fetched successfully: {'lat': 46.198460263054265, 'lon': 'nan', 'current_time': '02/23/24:01:19:28', 'air_temperature': 56.95358130449186}
error, missing data
Data fetched successfully: {'lat': 46.759773250364695, 'lon': -94.24279836343821, 'current_time': '02/23/24:01:19:28', 'air_temperature': 42.899186554460314}
no missing data, continuing
Data within Minnesota vertically!
Data within Minnesota horizontally!
Temperature is slay!
Time is slay!
Data fetched successfully: {'lat': 46.7912

In [66]:
print(final_data)
print(total_calls)
print(errors)

[{'lat': 46.792283865223546, 'lon': -93.67735204559952, 'current_time': '02/23/24:01:18:50', 'air_temperature': 19.55294158064669}, {'lat': 46.22123560128494, 'lon': -94.8771750165459, 'current_time': '02/23/24:01:18:51', 'air_temperature': 37.50829940509462}, {'lat': 46.31469456938011, 'lon': -94.1106738609918, 'current_time': '02/23/24:01:18:52', 'air_temperature': 21.93054309139302}, {'lat': 47.57702978511403, 'lon': -93.08449750890286, 'current_time': '02/23/24:01:18:53', 'air_temperature': 31.902841642894845}, {'lat': 45.840253968832926, 'lon': -95.1993171481796, 'current_time': '02/23/24:01:18:53', 'air_temperature': 21.784035603564035}, {'lat': 46.865327158727034, 'lon': -93.74648354536856, 'current_time': '02/23/24:01:18:54', 'air_temperature': 43.263553606702054}, {'lat': 48.21722313196884, 'lon': -94.6308505900213, 'current_time': '02/23/24:01:18:54', 'air_temperature': 38.92787214877515}, {'lat': 47.653800570714516, 'lon': -94.86711562749566, 'current_time': '02/23/24:01:18: